### Peer Review by Jenna White  
At the top of this notebook I pull in all of your code so I can make comments below!

In [23]:
import requests
import pandas as pd 
import numpy as np
import json

# grabbing data for Maternal Mortality Rate (MMR) 'MDG_0000000026' 
# Values are per 100K births         
C = requests.get('http://apps.who.int/gho/athena/api/GHO/MDG_0000000026.json')

#Converting data into json format
data = C.json()
data.keys()

[u'attribute', u'fact', u'dimension', u'copyright', u'dataset']

In [24]:
#Step 1: Creating a data frane that contains the MMR (per 100,000 births) values 
fact = data['fact']
MMRfact = pd.DataFrame(fact)   


In [25]:
#Step 2: extract the MMR out of MMRfact.value column and concat 
#extracting values from the MMR dataframe
data3 = list(MMRfact.value)
values = pd.DataFrame(data=data3)

In [26]:
data4=list(MMRfact.Dim)

In [27]:
dim_values = pd.DataFrame(data=data4)

In [28]:
#concat values onto the MMRfact data frame 
MMRfact = pd.concat([MMRfact, values], axis=1)
MMRfact.head()

#Step 3: extract the country code('COUNTRY'), region ('REGION') and year ('YEAR') from the MMRfact.Dim column
def extract(x):
    for col in x:
        if col['category'] == 'COUNTRY':
            return col['code'] 

MMRfact['COUNTRY'] = [extract(row) for row in MMRfact['Dim']]

def extract(x):
    for col in x:
        if col['category'] == 'YEAR':
            return col['code'] 

MMRfact['YEAR'] = [extract(row) for row in MMRfact['Dim']]

MMRfact['REGION'] = [extract(row) for row in MMRfact['Dim']]

def extract(x):
    for col in x:
        if col['category'] == 'REGION':
            return col['code'] 

MMRfact['REGION'] = [extract(row) for row in MMRfact['Dim']]


# simplify dataframe, only including relevant columns, changing COUNTRY to ISO (three letter code)           
MMRfact.drop(['Dim', 'dataset', 'effective_date', 'end_date', 'fact_id', 'published', 'value', 'display', 'high', 'low'], axis=1, inplace=True)
MMRfact = MMRfact.rename(columns = {'COUNTRY' : 'ISO', 'numeric' : 'MMR100K'})

In [29]:
# Step 4: load or extract a country:ISO dictionary from data['Dimension'] 
MMRdim = data['dimension']

#creating a country and ISO code dictionary 
country = {

    }

for row in MMRdim:
    for element in row['code']:
        ISO = None
        display = element['display']
        for x in element['attr']:
            if x['category'] == 'ISO': 
                ISO = x['value']
        country[ISO] = display    
#print country

In [30]:
#step 5: add the country names into the MMRfact data frame, rename MMR, extract info for relevant year

def name(x):
    for b in country:
        if x == b:
            return country[b]

MMRfact['COUNTRY'] = [name(a) for a in MMRfact['ISO']]

#rename MMR
MMR = MMRfact

#extract info only for 2015 
MMR['YEAR'] = pd.to_numeric(MMR['YEAR'])
MMR = MMR[MMR['YEAR'] == 2015]

-----

#### Feedback:

I liked the histogram you used to represent the varaince between countries in a region - very helpful for highlighting that Africa is not a binary situation. I think another area you could dig into is the EMR region. Super low median mortality rate but then some very high outliers.  

I pulled out that region below and sorted by MMR:


In [21]:
MMR[MMR['REGION']=='EMR'].sort_values(by=['MMR100K'], ascending=False)

,ISO,YEAR,REGION,MMR100K,COUNTRY
327,SOM,2015,EMR,732.0,Somalia
320,AFG,2015,EMR,396.0,Afghanistan
496,YEM,2015,EMR,385.0,Yemen
53,SDN,2015,EMR,311.0,Sudan
38,DJI,2015,EMR,229.0,Djibouti
564,PAK,2015,EMR,178.0,Pakistan
332,None,2015,EMR,166.0,Global
156,MAR,2015,EMR,121.0,Morocco
246,SYR,2015,EMR,68.0,Syrian Arab Republic
248,TUN,2015,EMR,62.0,Tunisia


The item that jumped out to me about the few at the top that are so much higher than the others is the **prevalance of war** in the region over the last decade.  

I looked through the WHO site and found that there is a WHO Mortality Database that has death rates by cause by country. It doesn't look like there is a cut and dry "War" option, but maybe there is a stand-in that you could use. Looks like there might be an external factor grouping: http://www.who.int/healthinfo/mortality_data/en/ -- you also listed in 4.1 a death rate feature, so maybe you could use that one



This may also help determine why some African countries are so different from others.


**Other Features**:
- Out of 4.1C I like Death Rate (as mentioned above), water/sanitaion, and fertility rate
- Also could potentially check out "Nursing and midwifery personnel density" instead of Health Service Coverage http://apps.who.int/gho/data/node.imr.HRH_33?lang=en

**Model Build**  
I agree with your note that a binary response variable is very blunt (I'm sure it has taken a while to get all of this data into a useable form - great job with that!!). What you could try, as a middle between binary and a rate, is categorizing MMRs into quarterly percentiles - may be more interesting a result

**Findings and Conclusions**  
I'd be interested in seeing the model coefficients so you could see how important they are when they are all in play (vs each individually)  - if you build a random forest you can use treereg.feature_importance_ 

Also ensembling is probably something you will want to incorporate to this as well.

**Overall, awesome job with this! Great job with the data parsing, and I think that this will be a nicely funcitoning model for the final project!**